In [ ]:
#!/usr/bin/env python3"""PROMETHEUS v3.0 - AIMO3 MATHEMATICAL OLYMPIAD SOLVER====================================================CRITICAL FIX: v2 scored 0/50 because vLLM import failed (scipy dependency).This version has bulletproof infrastructure validation and 3-tier fallback.Inference Tiers:  TIER 1: vLLM (fastest) - preferred  TIER 2: Transformers (reliable fallback)  TIER 3: Sympy-only (emergency)Author: Ryan J Cardwell + Claude Opus 4"""import sysimport osimport timeimport gcimport warningswarnings.filterwarnings('ignore')print("=" * 80)print("PROMETHEUS v3.0 - INFRASTRUCTURE VALIDATION")print("=" * 80)# Critical timestampsSTART_TIME = time.time()TOTAL_RUNTIME = 5 * 60 * 60 - 300  # 5 hours minus 5 min bufferCUTOFF_TIME = START_TIME + TOTAL_RUNTIMEPANIC_TIME = 300SEED = 42import randomrandom.seed(SEED)# Infrastructure statusINFRA_STATUS = {    "scipy_ok": False,    "vllm_ok": False,    "transformers_ok": False,    "sympy_ok": False,    "torch_ok": False,    "model_path": None,    "inference_tier": None,}# TIER 0: Core Dependenciesprint("\n[TIER 0] Core Dependencies...")import numpy as npnp.random.seed(SEED)print("  ✓ numpy")import refrom typing import List, Dict, Tuple, Optional, Any, NamedTuplefrom collections import Counter, defaultdictfrom dataclasses import dataclass, fieldfrom enum import Enumimport mathimport jsonimport tempfileimport subprocessfrom concurrent.futures import ThreadPoolExecutor, as_completedprint("  ✓ stdlib")# TIER 1: Scipyprint("\n[TIER 1] Scipy...")try:    import scipy    from scipy.optimize import linear_sum_assignment    INFRA_STATUS["scipy_ok"] = True    print(f"  ✓ scipy {scipy.__version__}")except ImportError as e:    print(f"  ⚠ scipy not available: {e}")    try:        subprocess.check_call([sys.executable, "-m", "pip", "install", "scipy", "-q"])        import scipy        from scipy.optimize import linear_sum_assignment        INFRA_STATUS["scipy_ok"] = True        print(f"  ✓ scipy {scipy.__version__} (installed)")    except:        INFRA_STATUS["scipy_ok"] = False# TIER 2: Torchprint("\n[TIER 2] PyTorch...")try:    import torch    INFRA_STATUS["torch_ok"] = True    print(f"  ✓ torch {torch.__version__}")    if torch.cuda.is_available():        print(f"    GPU: {torch.cuda.get_device_name(0)}")        print(f"    VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")except ImportError as e:    print(f"  ✗ torch: {e}")    INFRA_STATUS["torch_ok"] = False# TIER 3: vLLMprint("\n[TIER 3] vLLM...")os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"try:    from vllm import LLM, SamplingParams    INFRA_STATUS["vllm_ok"] = True    print("  ✓ vllm")except Exception as e:    print(f"  ⚠ vllm: {e}")    INFRA_STATUS["vllm_ok"] = False# TIER 4: Transformersprint("\n[TIER 4] Transformers...")try:    from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig    INFRA_STATUS["transformers_ok"] = True    print("  ✓ transformers")except ImportError as e:    print(f"  ⚠ transformers: {e}")    INFRA_STATUS["transformers_ok"] = False# TIER 5: Sympyprint("\n[TIER 5] Sympy...")try:    import sympy    from sympy import symbols, solve, simplify, Rational, Integer    from sympy.ntheory import factorint, divisors, totient, isprime    INFRA_STATUS["sympy_ok"] = True    print(f"  ✓ sympy {sympy.__version__}")except ImportError as e:    print(f"  ⚠ sympy: {e}")    INFRA_STATUS["sympy_ok"] = False# TIER 6: Model Discoveryprint("\n[TIER 6] Model Discovery...")MODEL_PATHS = [    "/kaggle/input/qwen-72b-math-int4",    "/kaggle/input/d/ryancardwell/qwen-72b-math-int4",    "/kaggle/input/qwen-72b-math-nf4",    "/kaggle/input/qwen2-5-math-72b-instruct",    "/kaggle/input/deepseek-coder-v2/transformers/deepseek-coder-v2-lite-instruct/1",]def find_model():    import glob    for p in MODEL_PATHS:        if os.path.exists(p):            if os.path.exists(os.path.join(p, "config.json")):                return p            configs = glob.glob(f"{p}/**/config.json", recursive=True)            if configs:                return os.path.dirname(configs[0])    if os.path.exists("/kaggle/input"):        for root, dirs, files in os.walk("/kaggle/input"):            if "config.json" in files and any(f.endswith('.safetensors') for f in files):                return root    return NoneINFRA_STATUS["model_path"] = find_model()if INFRA_STATUS["model_path"]:    print(f"  ✓ Model: {INFRA_STATUS['model_path']}")else:    print("  ⚠ No model found")# Determine inference tierprint("\n[INFERENCE TIER]")if INFRA_STATUS["vllm_ok"] and INFRA_STATUS["model_path"] and INFRA_STATUS["scipy_ok"]:    INFRA_STATUS["inference_tier"] = "vllm"    print("  → TIER 1: vLLM")elif INFRA_STATUS["transformers_ok"] and INFRA_STATUS["model_path"] and INFRA_STATUS["torch_ok"]:    INFRA_STATUS["inference_tier"] = "transformers"    print("  → TIER 2: Transformers")elif INFRA_STATUS["sympy_ok"]:    INFRA_STATUS["inference_tier"] = "sympy"    print("  → TIER 3: Sympy-only")else:    INFRA_STATUS["inference_tier"] = "panic"    print("  → TIER 4: PANIC")print("\n" + "=" * 80)print(f"STATUS: {INFRA_STATUS['inference_tier'].upper()}")print("=" * 80)

In [ ]:
# CELL 1: CONSTANTS AND TYPESprint("\n[1/8] Constants and Types...")ANSWER_MIN = 0ANSWER_MAX = 999999FALLBACK_ANSWER = 42class ProblemType(Enum):    NUMBER_THEORY = "number_theory"    COMBINATORICS = "combinatorics"    GEOMETRY = "geometry"    ALGEBRA = "algebra"    MIXED = "mixed"@dataclassclass ProblemProfile:    primary_type: ProblemType    modulo_target: Optional[int] = None    constraints: List[str] = field(default_factory=list)    is_enumerable: bool = False    enum_bound: Optional[int] = None@dataclassclass AnswerCandidate:    value: int    code: str    strategy: str    kolmogorov_weight: float = 1.0    confidence: float = 0.5print("  ✓ Types loaded")

In [ ]:
# CELL 2: PROMETHEUS CORE THEORY (10 INSIGHTS)print("\n[2/8] PROMETHEUS Core Theory...")def compute_kolmogorov_weight(code_length: int) -> float:    """Insight 1: Shorter code = higher confidence."""    if code_length <= 0:        return 0.5    return min(1.5, max(0.3, 200.0 / (code_length + 100)))def value_clustering(candidates: List[AnswerCandidate], threshold: float = 0.05) -> Dict[int, List[AnswerCandidate]]:    """Insight 2: Value clustering (88% error reduction)."""    if not candidates:        return {}        parent = {c.value: c.value for c in candidates}        def find(x):        if parent[x] != x:            parent[x] = find(parent[x])        return parent[x]        def union(x, y):        px, py = find(x), find(y)        if px != py:            parent[px] = py        values = [c.value for c in candidates]    for i, v1 in enumerate(values):        for v2 in values[i+1:]:            if v1 == v2:                union(v1, v2)            elif v1 > 0 and v2 > 0:                rel_diff = abs(v1 - v2) / max(abs(v1), abs(v2))                if rel_diff < threshold:                    union(v1, v2)        clusters = defaultdict(list)    for c in candidates:        clusters[find(c.value)].append(c)    return dict(clusters)BENFORD = {1: 0.301, 2: 0.176, 3: 0.125, 4: 0.097, 5: 0.079, 6: 0.067, 7: 0.058, 8: 0.051, 9: 0.046}def benford_score(value: int) -> float:    """Insight 3: Benford's Law scoring."""    if value <= 0:        return 0.5    leading = int(str(abs(value))[0])    return 0.5 + BENFORD.get(leading, 0.1)def seed_amplification(candidates: List[AnswerCandidate]) -> float:    """Insight 4: Seed amplification (μ > 1)."""    if len(candidates) < 2:        return 1.0    values = [c.value for c in candidates]    counts = Counter(values)    if not counts:        return 1.0    max_count = max(counts.values())    return 1.0 + (max_count - 1) * 0.15def select_answer_prometheus(candidates: List[AnswerCandidate], mod_target: Optional[int] = None) -> int:    """Master PROMETHEUS selection algorithm."""    if not candidates:        return FALLBACK_ANSWER        if mod_target and mod_target > 0:        for c in candidates:            c.value = c.value % mod_target        clusters = value_clustering(candidates)    if not clusters:        return FALLBACK_ANSWER        cluster_scores = {}    for center, members in clusters.items():        mass = len(members)        avg_kolmo = sum(c.kolmogorov_weight for c in members) / len(members)        strategies = set(c.strategy for c in members)        integration = 1.0 + 0.2 * (len(strategies) - 1)        benford = benford_score(center)        amp = seed_amplification(members)        score = mass * avg_kolmo * integration * benford * amp        cluster_scores[center] = score        best_value = max(cluster_scores, key=cluster_scores.get)        if best_value in [0, 1] and len(cluster_scores) > 1:        alternatives = [v for v in cluster_scores if v not in [0, 1]]        if alternatives:            best_value = max(alternatives, key=lambda x: cluster_scores[x])        return int(best_value) % (ANSWER_MAX + 1)TYPE_HINTS = {    ProblemType.NUMBER_THEORY: "Use modular arithmetic, factorization, divisibility.",    ProblemType.COMBINATORICS: "Count systematically. Use inclusion-exclusion.",    ProblemType.GEOMETRY: "Use coordinates, similar triangles, or trigonometry.",    ProblemType.ALGEBRA: "Solve equations systematically. Check special cases.",    ProblemType.MIXED: "Combine techniques. Look for hidden structure.",}print("  ✓ PROMETHEUS Core Theory loaded")

In [ ]:
# CELL 3: PROBLEM CLASSIFIERprint("\n[3/8] Problem Classifier...")NUMBER_THEORY_PATTERNS = [r'divis\w*', r'prime', r'factor', r'gcd', r'lcm', r'modulo?', r'remainder', r'coprime', r'\\bmod\\b']COMBINATORICS_PATTERNS = [r'how many', r'count', r'number of ways', r'permut', r'combin', r'arrange', r'choose', r'select']GEOMETRY_PATTERNS = [r'triangle', r'circle', r'square', r'rectangle', r'polygon', r'angle', r'area', r'perimeter', r'radius']ALGEBRA_PATTERNS = [r'equation', r'polynomial', r'root', r'solution', r'solve', r'variable', r'function', r'sequence']def classify_problem(question: str) -> ProblemProfile:    q = question.lower()    scores = {        ProblemType.NUMBER_THEORY: sum(1 for p in NUMBER_THEORY_PATTERNS if re.search(p, q)),        ProblemType.COMBINATORICS: sum(1 for p in COMBINATORICS_PATTERNS if re.search(p, q)),        ProblemType.GEOMETRY: sum(1 for p in GEOMETRY_PATTERNS if re.search(p, q)),        ProblemType.ALGEBRA: sum(1 for p in ALGEBRA_PATTERNS if re.search(p, q)),    }    max_score = max(scores.values())    primary = ProblemType.MIXED if max_score == 0 else max(scores, key=scores.get)        modulo_target = None    for pattern in [r'modulo?\s+(\d+)', r'remainder\s+when.*divided\s+by\s+(\d+)', r'mod\s+(\d+)']:        match = re.search(pattern, q)        if match:            modulo_target = int(match.group(1))            break        is_enumerable = bool(re.search(r'how many|count|number of', q))    return ProblemProfile(primary_type=primary, modulo_target=modulo_target, is_enumerable=is_enumerable)print("  ✓ Problem Classifier loaded")

In [ ]:
# CELL 4: CODE EXECUTION ENGINEprint("\n[4/8] Code Execution Engine...")STDLIB = '''import sys; sys.setrecursionlimit(20000)import math, numpy as npfrom itertools import combinations, permutations, productfrom collections import Counter, defaultdictfrom functools import lru_cache, reducefrom fractions import Fractiondef is_prime(n):    if n < 2: return False    if n < 4: return True    if n % 2 == 0: return False    for i in range(3, int(n**0.5) + 1, 2):        if n % i == 0: return False    return Truedef gcd(a, b):    while b: a, b = b, a % b    return adef lcm(a, b): return a * b // gcd(a, b)def C(n, k):    if k < 0 or k > n: return 0    return math.factorial(n) // (math.factorial(k) * math.factorial(n - k))try:    from sympy import *    from sympy.ntheory import factorint, divisors, totientexcept: pass'''SNOOP = '''_vars = dict(globals())for _v in [\'answer\', \'result\', \'ans\', \'res\', \'final\', \'output\', \'solution\', \'total\', \'count\']:    if _v in _vars and _vars[_v] is not None:        try: print(int(_vars[_v])); break        except: pass'''def extract_python_code(text: str) -> Optional[str]:    for pattern in [r\'```python\\s*\\n(.*?)```\', r\'```\\s*\\n(.*?)```\']:        matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)        if matches: return matches[-1].strip()    return Nonedef normalize_answer(raw: Any) -> Optional[int]:    if raw is None: return None    try:        s = str(raw).replace(\',\', \'\').replace(\' \', \'\').strip()        s = re.sub(r\'[^0-9.eE+-]\', \'\', s)        if not s: return None        val = float(s)        if val != val or abs(val) == float(\'inf\'): return None        result = int(round(val))        return abs(result) % (ANSWER_MAX + 1)    except: return Nonedef execute_code(code: str, timeout: int = 15) -> Tuple[Optional[int], str]:    if not code: return None, ""    has_print = \'print(\' in code    full = STDLIB + "\\n" + code + ("" if has_print else "\\n" + SNOOP)    try:        with tempfile.NamedTemporaryFile(mode=\'w\', suffix=\'.py\', delete=False) as f:            f.write(full)            f.flush()            result = subprocess.run([\'python\', f.name], capture_output=True, text=True, timeout=timeout)            os.unlink(f.name)            if result.returncode == 0 and result.stdout.strip():                numbers = re.findall(r\'-?\\d+\\.?\\d*\', result.stdout)                if numbers: return normalize_answer(numbers[-1]), code    except: pass    return None, codedef extract_boxed(text: str) -> Optional[int]:    for pattern in [r\'\\\\boxed\\{(\\d+)\\}\', r\'[Aa]nswer\\s*[=:]\\s*(\\d+)\']:        matches = re.findall(pattern, text, re.IGNORECASE)        if matches: return normalize_answer(matches[-1])    nums = re.findall(r\'\\b(\\d+)\\b\', text[-500:])    for n in reversed(nums):        v = normalize_answer(n)        if v and 1 < v < 100000: return v    return Noneprint("  ✓ Code Execution Engine loaded")

In [ ]:
# CELL 5: MODEL LOADING (3-TIER)print("\n[5/8] Model Loading...")LLM_MODEL = NoneTOKENIZER = NoneHF_MODEL = NoneHF_TOKENIZER = NoneSTRATEGY_PROMPTS = {    'pot_algorithmic': "Write self-contained Python. Use int() not float. Print ONE integer.",    'pot_sympy': "Use sympy for EXACT arithmetic. Use Rational(a,b) NOT a/b. Print ONE integer.",    'pot_bruteforce': "Systematically enumerate. Budget: 10^7 iterations max. Print ONE integer.",    'cot': "Think step-by-step. Output final answer in \\boxed{}.",}TEMP_START = 0.7TEMP_MIN = 0.3TEMP_DECAY = 0.85POT_STOP = ["```output", "```\nOutput", "# Output", "\n\n\n", "<|im_end|>"]COT_STOP = ["</think>", "\n\nQuestion:", "<|im_end|>"]def create_prompt(question: str, strategy: str, profile: ProblemProfile) -> str:    system = STRATEGY_PROMPTS.get(strategy, STRATEGY_PROMPTS['pot_algorithmic'])    hint = TYPE_HINTS.get(profile.primary_type, "")    mod_note = f"\nAnswer must be mod {profile.modulo_target}." if profile.modulo_target else ""    return f"""<|im_start|>system{system} {hint}{mod_note}<|im_end|><|im_start|>user{question}<|im_end|><|im_start|>assistant"""# TIER 1: vLLMif INFRA_STATUS["inference_tier"] == "vllm":    print(f"  Loading vLLM from {INFRA_STATUS['model_path']}...")    try:        LLM_MODEL = LLM(            model=INFRA_STATUS["model_path"],            tensor_parallel_size=1,            gpu_memory_utilization=0.92,            trust_remote_code=True,            max_model_len=8192,            enforce_eager=True,            seed=SEED,        )        TOKENIZER = LLM_MODEL.get_tokenizer()        print("  ✓ vLLM loaded!")    except Exception as e:        print(f"  ✗ vLLM failed: {e}")        if INFRA_STATUS["transformers_ok"]:            INFRA_STATUS["inference_tier"] = "transformers"        else:            INFRA_STATUS["inference_tier"] = "sympy"# TIER 2: Transformersif INFRA_STATUS["inference_tier"] == "transformers":    print(f"  Loading Transformers from {INFRA_STATUS['model_path']}...")    try:        bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)        HF_TOKENIZER = AutoTokenizer.from_pretrained(INFRA_STATUS["model_path"], trust_remote_code=True)        HF_MODEL = AutoModelForCausalLM.from_pretrained(INFRA_STATUS["model_path"], quantization_config=bnb_config, device_map="auto", trust_remote_code=True)        print("  ✓ Transformers loaded!")    except Exception as e:        print(f"  ✗ Transformers failed: {e}")        INFRA_STATUS["inference_tier"] = "sympy"if INFRA_STATUS["inference_tier"] == "sympy":    print("  Using Sympy-only mode")print(f"\n  Final tier: {INFRA_STATUS['inference_tier'].upper()}")

In [ ]:
# CELL 6: GENERATION FUNCTIONSprint("\n[6/8] Generation Functions...")def vllm_generate(prompts: List[str], temp: float = 0.6, stop: List[str] = POT_STOP) -> List[str]:    if not LLM_MODEL: return [""] * len(prompts)    params = SamplingParams(temperature=max(0.1, temp), top_p=0.9, max_tokens=4096, stop=stop)    outputs = LLM_MODEL.generate(prompts, sampling_params=params)    return [o.outputs[0].text for o in outputs]def transformers_generate(prompts: List[str], temp: float = 0.6, max_tokens: int = 2048) -> List[str]:    if not HF_MODEL or not HF_TOKENIZER: return [""] * len(prompts)    results = []    for prompt in prompts:        try:            inputs = HF_TOKENIZER(prompt, return_tensors="pt").to(HF_MODEL.device)            with torch.no_grad():                outputs = HF_MODEL.generate(**inputs, max_new_tokens=max_tokens, temperature=max(0.1, temp), top_p=0.9, do_sample=True, pad_token_id=HF_TOKENIZER.eos_token_id)            result = HF_TOKENIZER.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)            results.append(result)        except: results.append("")    return resultsdef batch_generate(prompts: List[str], mode: str = 'pot', temp: float = 0.6) -> List[str]:    stop = POT_STOP if mode == 'pot' else COT_STOP    if INFRA_STATUS["inference_tier"] == "vllm": return vllm_generate(prompts, temp, stop)    elif INFRA_STATUS["inference_tier"] == "transformers": return transformers_generate(prompts, temp)    return [""] * len(prompts)def process_pot_outputs(outputs: List[str], strategies: List[str]) -> List[AnswerCandidate]:    candidates = []    with ThreadPoolExecutor(max_workers=8) as executor:        futures = {}        for i, out in enumerate(outputs):            code = extract_python_code(out)            if code:                futures[executor.submit(execute_code, code)] = (i, code, strategies[i] if i < len(strategies) else 'pot')        for future in as_completed(futures):            i, code, strat = futures[future]            try:                ans, final_code = future.result()                if ans is not None:                    candidates.append(AnswerCandidate(value=ans, code=final_code, strategy=strat, kolmogorov_weight=compute_kolmogorov_weight(len(final_code))))            except: pass    return candidatesdef process_cot_outputs(outputs: List[str]) -> List[AnswerCandidate]:    candidates = []    for out in outputs:        ans = extract_boxed(out)        if ans is not None:            candidates.append(AnswerCandidate(value=ans, code="", strategy="cot", kolmogorov_weight=1.0))    return candidatesprint("  ✓ Generation Functions loaded")

In [ ]:
# CELL 7: SYMPY SOLVER (TIER 3 FALLBACK)print("\n[7/8] Sympy Solver...")def sympy_solve_problem(question: str, profile: ProblemProfile) -> List[AnswerCandidate]:    if not INFRA_STATUS["sympy_ok"]: return []    candidates = []    try:        numbers = [int(x) for x in re.findall(r\'\\b(\\d+)\\b\', question) if 0 < int(x) < 1000000]                if profile.primary_type == ProblemType.NUMBER_THEORY:            if len(numbers) >= 2 and (\'gcd\' in question.lower() or \'greatest common\' in question.lower()):                result = math.gcd(*numbers[:2])                candidates.append(AnswerCandidate(value=result, code="gcd", strategy="sympy_gcd", kolmogorov_weight=1.2))            if len(numbers) >= 2 and (\'lcm\' in question.lower() or \'least common\' in question.lower()):                result = math.lcm(*numbers[:2])                candidates.append(AnswerCandidate(value=result % (ANSWER_MAX + 1), code="lcm", strategy="sympy_lcm", kolmogorov_weight=1.2))            if numbers and \'divisor\' in question.lower():                n = numbers[0]                if n < 10**12:                    factors = factorint(n)                    num_divisors = 1                    for exp in factors.values(): num_divisors *= (exp + 1)                    candidates.append(AnswerCandidate(value=num_divisors, code="divisors", strategy="sympy_factor", kolmogorov_weight=1.2))                elif profile.primary_type == ProblemType.COMBINATORICS:            if len(numbers) >= 2:                n, k = numbers[0], numbers[1]                if \'permut\' in question.lower() and n <= 20 and k <= n:                    result = math.perm(n, k)                    candidates.append(AnswerCandidate(value=result % (ANSWER_MAX + 1), code="perm", strategy="sympy_perm", kolmogorov_weight=1.2))                elif (\'combin\' in question.lower() or \'choose\' in question.lower()) and n <= 100 and k <= n:                    result = math.comb(n, k)                    candidates.append(AnswerCandidate(value=result % (ANSWER_MAX + 1), code="comb", strategy="sympy_comb", kolmogorov_weight=1.2))                if profile.modulo_target:            for c in candidates: c.value = c.value % profile.modulo_target    except: pass    return candidatesprint("  ✓ Sympy Solver loaded")

In [ ]:
# CELL 8: PROMETHEUS SOLVERprint("\n[8/8] PROMETHEUS Solver...")STRATEGY_WEIGHTS = {    ProblemType.NUMBER_THEORY: ['pot_sympy', 'pot_sympy', 'pot_algorithmic', 'pot_bruteforce'],    ProblemType.COMBINATORICS: ['pot_bruteforce', 'pot_bruteforce', 'pot_algorithmic', 'pot_sympy'],    ProblemType.GEOMETRY: ['pot_sympy', 'pot_algorithmic', 'pot_algorithmic', 'cot'],    ProblemType.ALGEBRA: ['pot_sympy', 'pot_sympy', 'pot_algorithmic', 'cot'],    ProblemType.MIXED: ['pot_algorithmic', 'pot_sympy', 'pot_bruteforce', 'cot'],}PROBLEM_COUNT = 0SOLVED_COUNT = 0AAR_LOG = defaultdict(int)def prometheus_refine(candidates: List[AnswerCandidate], mod_target: Optional[int] = None, iterations: int = 3) -> int:    if not candidates: return FALLBACK_ANSWER    current_candidates = candidates[:]    for i in range(iterations):        current_answer = select_answer_prometheus(current_candidates, mod_target)        k = max(1, len(current_candidates) // 4)        seed_candidates = [AnswerCandidate(value=current_answer, code="seed", strategy="prometheus", kolmogorov_weight=1.5) for _ in range(k)]        current_candidates = current_candidates + seed_candidates    return select_answer_prometheus(current_candidates, mod_target)def toroidal_vote(samples: List[int], mod: int) -> Tuple[int, float]:    if not samples or mod <= 0: return 0, 0.1    normalized = [s % mod for s in samples]    angles = [2 * math.pi * m / mod for m in normalized]    sin_sum = sum(math.sin(a) for a in angles)    cos_sum = sum(math.cos(a) for a in angles)    mean_angle = math.atan2(sin_sum, cos_sum)    center = int(round(mean_angle * mod / (2 * math.pi))) % mod    R = math.sqrt(sin_sum**2 + cos_sum**2) / len(samples)    return center, min(0.95, max(0.1, R))def panic_guess(question: str, profile: ProblemProfile) -> int:    AAR_LOG['panic'] += 1    numbers = [int(x) for x in re.findall(r\'\\b(\\d+)\\b\', question) if 0 < int(x) < 100000]    if profile.modulo_target and numbers: return numbers[0] % profile.modulo_target    if numbers:        numbers.sort()        return numbers[len(numbers) // 2]    return FALLBACK_ANSWERdef solve_problem(question: str) -> int:    global PROBLEM_COUNT, SOLVED_COUNT    PROBLEM_COUNT += 1    time_remaining = CUTOFF_TIME - time.time()        print(f"\n{'='*60}")    print(f"[Problem {PROBLEM_COUNT}] Time: {time_remaining/60:.1f} min")        if time_remaining < PANIC_TIME:        profile = classify_problem(question)        print(f"  ⚠️ PANIC MODE")        return panic_guess(question, profile)        profile = classify_problem(question)    print(f"  Type: {profile.primary_type.value} | Mod: {profile.modulo_target}")        strategies = STRATEGY_WEIGHTS.get(profile.primary_type, STRATEGY_WEIGHTS[ProblemType.MIXED])    all_candidates = []    current_temp = TEMP_START        # Phase 1: LLM Generation    if INFRA_STATUS["inference_tier"] in ["vllm", "transformers"]:        print(f"  Phase 1: {len(strategies)} samples (T={current_temp:.2f})")        prompts = [create_prompt(question, s, profile) for s in strategies]        outputs = batch_generate(prompts, 'pot', current_temp)        candidates = process_pot_outputs(outputs, strategies)        all_candidates.extend(candidates)        print(f"    → {len(candidates)} answers: {[c.value for c in candidates]}")                if len(candidates) >= 2:            values = [c.value for c in candidates]            counts = Counter(values)            best, count = counts.most_common(1)[0]            if count / len(values) >= 0.6:                print(f"  ✓ CONSENSUS: {best}")                SOLVED_COUNT += 1                return int(best)                # Phase 2: Expansion        time_remaining = CUTOFF_TIME - time.time()        if time_remaining > 180 and len(all_candidates) < 4:            current_temp = max(TEMP_MIN, current_temp * TEMP_DECAY)            print(f"  Phase 2: Expansion (T={current_temp:.2f})")            prompts2 = [create_prompt(question, s, profile) for s in strategies]            outputs2 = batch_generate(prompts2, 'pot', current_temp)            candidates2 = process_pot_outputs(outputs2, strategies)            all_candidates.extend(candidates2)                # Phase 3: CoT        time_remaining = CUTOFF_TIME - time.time()        if len(all_candidates) < 3 and time_remaining > 120:            print(f"  Phase 3: CoT fallback")            cot_prompts = [create_prompt(question, 'cot', profile) for _ in range(2)]            cot_outputs = batch_generate(cot_prompts, 'cot', 0.4)            cot_candidates = process_cot_outputs(cot_outputs)            all_candidates.extend(cot_candidates)        # Phase 4: Sympy    print(f"  Phase 4: Sympy solver")    sympy_candidates = sympy_solve_problem(question, profile)    all_candidates.extend(sympy_candidates)    if sympy_candidates:        print(f"    → {len(sympy_candidates)} sympy: {[c.value for c in sympy_candidates]}")        if not all_candidates:        print("  ⚠️ No answers - panic guess")        AAR_LOG['no_answers'] += 1        return panic_guess(question, profile)        # PROMETHEUS Refinement    final = prometheus_refine(all_candidates, profile.modulo_target, iterations=3)        # Toroidal verification    if profile.modulo_target:        values = [c.value for c in all_candidates]        tor_ans, tor_conf = toroidal_vote(values, profile.modulo_target)        if tor_conf > 0.7 and tor_ans != final:            print(f"    Toroidal: {final} → {tor_ans}")            final = tor_ans        print(f"  ✓ FINAL: {final}")    SOLVED_COUNT += 1    gc.collect()    if INFRA_STATUS["torch_ok"] and torch.cuda.is_available(): torch.cuda.empty_cache()    return finalprint("  ✓ PROMETHEUS Solver loaded")print("\n" + "=" * 80)print(f"PROMETHEUS v3.0 READY | Tier: {INFRA_STATUS['inference_tier'].upper()}")print("=" * 80)

In [ ]:
# CELL 9: KAGGLE API INTERFACEimport aimo_3_gatewaydef predict_for_question(question: str) -> int:    """Kaggle API entry point."""    return solve_problem(question)# Register with Kaggleinference_server = aimo_3_gateway.AIMO3InferenceServer(predict_for_question)inference_server.serve()